# 트랜스포머야!
- 트랜스포머지.
- 일단 모델 설계부터 먼저 해볼까?

In [9]:
import torch

import torch.nn as nn

device = torch.device( "cuda" if torch.cuda.is_available else "mps" if torch.mps.is_available() else "cpu")


## 데이터 가져오기

In [34]:
from datasets import load_dataset

# 스트리밍 모드로 데이터셋 로드
dataset = load_dataset("cornell-movie-dialog/cornell_movie_dialog", streaming=True)

# 'train' 스플릿에서 처음 20,000개의 샘플만 가져오기
reduced_dataset = dataset['train'].take(20000)

# 리스트로 변환 (선택사항)
reduced_dataset_list = list(reduced_dataset)

print(f"Reduced dataset size: {len(reduced_dataset_list)}")


/opt/anaconda3/envs/secondllm/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for cornell-movie-dialog/cornell_movie_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cornell-movie-dialog/cornell_movie_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


KeyboardInterrupt: 

In [33]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-dialog/cornell_movie_dialog", trust_remote_code=True)


# dataset = load_dataset("imdb")
# train_dataset = dataset['train']
# test_dataset = dataset['test']

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # 'unsupervised' 데이터셋 제거
# del dataset['unsupervised']

# # 'train'과 'test' 데이터셋 크기 조정
# dataset['train'] = dataset['train'].select(range(10000))
# dataset['test'] = dataset['test'].select(range(10000))


/opt/anaconda3/envs/secondllm/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for cornell-movie-dialog/cornell_movie_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cornell-movie-dialog/cornell_movie_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/83097 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [3]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding="max_length", truncation=True)
    
    # 특정 필드만 float로 변환 (예: 'input_ids')
    if 'input_ids' in tokenized:
        tokenized['input_ids'] = [torch.tensor(ids, dtype=torch.float) for ids in tokenized['input_ids']]
    
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
from torch.utils.data import DataLoader

train_dataset = tokenized_datasets["train"].remove_columns(["text"])
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format("torch")

test_dataset = tokenized_datasets["test"].remove_columns(["text"])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format("torch")

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

## 모델 구성

### PositionalEncoding

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.seq_len = max_seq_len
        self.d_model = d_model
        self.positional_encoding = self.get_pos_encoding(max_seq_len, d_model)

    def get_angles(self, position, i, d_model):
        angles = position / torch.pow(10000, (2 * (i // 2)) / d_model)
        return angles

    def get_pos_encoding(self, max_seq_len, d_model):
        angle_tensor = self.get_angles(
            position=torch.arange(max_seq_len).unsqueeze(1),
            i=torch.arange(d_model).unsqueeze(0),
            d_model=torch.tensor(d_model, dtype=torch.float32)
        )
        positional_encoding = torch.zeros(max_seq_len, d_model, dtype=torch.float32)
        positional_encoding[:, 0::2] = torch.sin(angle_tensor[:, 0::2])
        positional_encoding[:, 1::2] = torch.cos(angle_tensor[:, 1::2])
        return positional_encoding

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.positional_encoding[:seq_len, :].unsqueeze(0).to(x.device)




In [12]:
### pe 실험
max_seq_len = 512
d_model = 512

x = torch.linspace(0, 1, max_seq_len).unsqueeze(0)
pe =  PositionalEncoding(max_seq_len=max_seq_len, d_model=d_model)
encoded_x = pe(x)

# encoded_x =  x +  pe[:x.size(0), :]
print(encoded_x.shape)



torch.Size([1, 512, 512])


### AttentionLayer

In [25]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, use_encoder_output=False, dropout_prob=0.1):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.use_encoder_output = use_encoder_output
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout_prob)  # 드롭아웃 추가
        self.layer_norm = nn.LayerNorm(d_model)
    
    def split_heads(self, x):
        batch_size, seq_length, _ = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    
    def forward(self, x, mask, encoder_x=None):
        batch_size, seq_length, _ = x.size()
        
        # Linear projections
        q = self.W_q(x)
        if self.use_encoder_output:
            k = self.W_k(encoder_x)
            v = self.W_v(encoder_x)
        else:
            k = self.W_k(x)
            v = self.W_v(x)
        
        # Split heads
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        # Scaled dot-product attention
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))
        if mask is not None:
            attention_scores += mask * (-1e9)
        
        attention_probs = torch.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)  # Dropout 적용

        context = torch.matmul(attention_probs, v)
        
        # Combine heads
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        output = self.W_o(context)
        output = self.dropout(output)  # Dropout 적용
        
        # Residual connection + Layer Norm
        return self.layer_norm(x + output)



### FFNNLayer

In [26]:
class FFNN(nn.Module):
    def __init__(self, d_model, d_ff, dropout_prob=0.1):
        super(FFNN, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.linear1 = nn.Linear(d_model, d_ff, bias=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)  # Dropout 추가
        self.linear2 = nn.Linear(d_ff, d_model, bias=True)

        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        output = self.linear1(x)
        output = self.relu(output)  # ReLU 적용
        output = self.dropout(output)  # Dropout 적용
        output = self.linear2(output)
        output = self.dropout(output)  # Dropout 적용

        return self.layer_norm(x + output)  # Residual Connection + Layer Norm



### encoder, decoder layer

In [27]:
import torch.nn as nn

class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, dropout_prob=0.1):
        super(EncoderLayer, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, d_model=d_model, dropout_prob=dropout_prob)  # Dropout 추가
        self.ffnn = FFNN(d_model=d_model, d_ff=d_ff, dropout_prob=dropout_prob)  # Dropout 추가

    def forward(self, x, mask):
        x = self.att(x, mask)
        x = self.ffnn(x)
        return x






In [31]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, num_heads, dropout_prob=0.1):
        super(DecoderLayer, self).__init__()
        self.att1 = MultiHeadAttention(num_heads=num_heads, d_model=d_model, dropout_prob=dropout_prob)  # Dropout 추가
        self.att2 = MultiHeadAttention(num_heads=num_heads, d_model=d_model, use_encoder_output=True, dropout_prob=dropout_prob)  # Dropout 추가
        self.ffnn = FFNN(d_model=d_model, d_ff=d_ff, dropout_prob=dropout_prob)  # Dropout 추가

    def forward(self, x, mask, encoded_x):
        if mask is None:
            n = x.size(1)
            look_ahead_mask = torch.triu(torch.ones(n, n), diagonal=1)
        else:
            look_ahead_mask = torch.logical_or(mask, torch.triu(torch.ones(n, n), diagonal=1))
        
        x = self.att1(x, look_ahead_mask)
        x = self.att2(x, mask, encoded_x=encoded_x)
        x = self.ffnn(x)
        return x


### 최종 모델

In [28]:
class MyTransformer(nn.Module):
    def __init__(self, max_seq_len, d_model, d_ff, num_heads, num_layers=6, dropout_prob=0.1):
        super(MyTransformer, self).__init__()
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, d_ff, num_heads, dropout_prob=dropout_prob) for _ in range(num_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, d_ff, num_heads, dropout_prob=dropout_prob) for _ in range(num_layers)
        ])
        self.pe = PositionalEncoding(max_seq_len=max_seq_len, d_model=d_model)

    def forward(self, src, tgt, src_mask, tgt_mask):
        src = self.pe(src)
        tgt = self.pe(tgt)

        # Encoder
        encoded_src = src
        for layer in self.encoder_layers:
            encoded_src = layer(encoded_src, src_mask)

        # Decoder
        for layer in self.decoder_layers:
            tgt = layer(tgt, tgt_mask, encoded_x=encoded_src)
        
        return tgt



## 시험해보기

In [32]:
from torch.optim import Adam
from torch.utils.data import DataLoader

model = MyTransformer(max_seq_len=512, d_model=512, d_ff=2048, num_heads=8, num_layers=6)
# 옵티마이저 설정
optimizer = Adam(model.parameters(), lr=0.001)

# 손실 함수 정의
criterion = nn.BCELoss()

# 학습 루프
model.to(device)

MyTransformer(
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (att): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (ffnn): FFNN(
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLayer(
      (att1): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bi

In [24]:


num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].float().to(device)
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    
    # 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids)
            predicted = (outputs.squeeze() > 0.5).long()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}")

TypeError: MyTransformer.forward() missing 3 required positional arguments: 'tgt', 'src_mask', and 'tgt_mask'